In [1]:
#from google.colab import drive
#drive.mount("/content/drive", force_remount=True)

## Import Dependencies

In [1]:
#file
import json
import xml.etree.ElementTree as ET
import os

#data science
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import numpy as np

#images
from PIL import Image
import cv2
import h5py
from skimage import util
from skimage.measure import label, regionprops
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
# from skimage.measure import compare_psnr updated to peak_signal_noise_ratio
# from skimage.measure import compare_ssim updated to structural_similarity

#stats
import math 
import random
import scipy.io as sio
import scipy
from scipy.ndimage import gaussian_filter, morphology

#torch modules
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import StepLR, MultiStepLR

#model
from sklearn import linear_model

#time
import argparse
from time import time

## Adjust And Format Image Files

In [2]:
def read_image(x):
    img_arr = np.array(Image.open(x))
    if len(img_arr.shape) == 2:  # grayscale
        img_arr = np.tile(img_arr, [3, 1, 1]).transpose(1, 2, 0)
    return img_arr

In [3]:
class RandomCrop(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):

        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        h, w = image.shape[:2]

        if isinstance(self.output_size, tuple):
            new_h = min(self.output_size[0], h)
            new_w = min(self.output_size[1], w)
            assert (new_h, new_w) == self.output_size
        else:
            crop_size = min(self.output_size, h, w)
            assert crop_size == self.output_size
            new_h = new_w = crop_size
        if gtcount > 0:
            mask = target > 0
            ch, cw = int(np.ceil(new_h / 2)), int(np.ceil(new_w / 2))
            mask_center = np.zeros((h, w), dtype=np.uint8)
            mask_center[ch:h-ch+1, cw:w-cw+1] = 1
            mask = (mask & mask_center)
            idh, idw = np.where(mask == 1)
            if len(idh) != 0:
                ids = random.choice(range(len(idh)))
                hc, wc = idh[ids], idw[ids]
                top, left = hc-ch, wc-cw
            else:
                top = np.random.randint(0, h-new_h+1)
                left = np.random.randint(0, w-new_w+1)
        else:
            top = np.random.randint(0, h-new_h+1)
            left = np.random.randint(0, w-new_w+1)

        image = image[top:top+new_h, left:left+new_w, :]
        target = target[top:top+new_h, left:left+new_w]

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [4]:
class RandomFlip(object):
    def __init__(self):
        pass

    def __call__(self, sample):
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        do_mirror = np.random.randint(2)
        if do_mirror:
            image = cv2.flip(image, 1)
            target = cv2.flip(target, 1)
        return {'image': image, 'target': target, 'gtcount': gtcount}

In [5]:
class Normalize(object):

    def __init__(self, scale, mean, std):
        self.scale = scale
        self.mean = mean
        self.std = std

    def __call__(self, sample):
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        image, target = image.astype('float32'), target.astype('float32')

        # pixel normalization
        image = (self.scale * image - self.mean) / self.std

        image, target = image.astype('float32'), target.astype('float32')

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [6]:
class ZeroPadding(object):
    def __init__(self, psize=32):
        self.psize = psize

    def __call__(self, sample):
        psize =  self.psize

        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        h,w = image.size()[-2:]
        ph,pw = (psize-h%psize),(psize-w%psize)
        print(ph,pw)

        (pl, pr) = (pw//2, pw-pw//2) if pw != psize else (0, 0)
        (pt, pb) = (ph//2, ph-ph//2) if ph != psize else (0, 0)
        if (ph!=psize) or (pw!=psize):
            tmp_pad = [pl, pr, pt, pb]
            print(tmp_pad)
            image = F.pad(image,tmp_pad)
            target = F.pad(target,tmp_pad)

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [7]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __init__(self):
        pass

    def __call__(self, sample):
        # swap color axis
        # numpy image: H x W x C
        # torch image: C X H X W
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        image = image.transpose((2, 0, 1))
        target = np.expand_dims(target, axis=2)
        target = target.transpose((2, 0, 1))
        image, target = torch.from_numpy(image), torch.from_numpy(target)
        return {'image': image, 'target': target, 'gtcount': gtcount}

In [8]:
class MaizeTasselDataset(Dataset):
    def __init__(self, data_dir, data_list, ratio, train=True, transform=None):
        self.data_dir = data_dir
        self.data_list = [name.split('\t') for name in open(data_list).read().splitlines()]
        self.ratio = ratio
        self.train = train
        self.transform = transform
        self.image_list = []
        
        # store images and generate ground truths
        self.images = {}
        self.targets = {}
        self.gtcounts = {}
        self.dotimages = {}

    def bbs2points(self, bbs):    
        points = []
        for bb in bbs:
            x1, y1, w, h = [float(b) for b in bb]
            x2, y2 = x1+w-1, y1+h-1
            x, y = np.round((x1+x2)/2).astype(np.int32), np.round((y1+y2)/2).astype(np.int32)
            points.append([x, y])
        return points

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        file_name = self.data_list[idx]
        self.image_list.append(file_name[0])
        if file_name[0] not in self.images:
            image = read_image(self.data_dir+file_name[0])
            annotation = sio.loadmat(self.data_dir+file_name[1])
            h, w = image.shape[:2]
            nh = int(np.ceil(h * self.ratio))
            nw = int(np.ceil(w * self.ratio))
            image = cv2.resize(image, (nw, nh), interpolation = cv2.INTER_CUBIC)
            target = np.zeros((nh, nw), dtype=np.float32)
            dotimage = image.copy()
            if annotation['annotation'][0][0][1] is not None:
                bbs = annotation['annotation'][0][0][1]
                gtcount = bbs.shape[0]
                pts = self.bbs2points(bbs)
                for pt in pts:
                    pt[0], pt[1] = int(pt[0] * self.ratio), int(pt[1] * self.ratio)
                    target[pt[1], pt[0]] = 1
                    cv2.circle(dotimage, (pt[0], pt[1]), int(24 * self.ratio) , (255, 0, 0), -1)
            else:
                gtcount = 0
            target = gaussian_filter(target, 80 * self.ratio)

            #plt.imshow(target, cmap=cm.jet)
            #plt.show()
            print(target.sum())

            self.images.update({file_name[0]:image})
            self.targets.update({file_name[0]:target})
            self.gtcounts.update({file_name[0]:gtcount})
            self.dotimages.update({file_name[0]:dotimage})

        
        sample = {
            'image': self.images[file_name[0]], 
            'target': self.targets[file_name[0]], 
            'gtcount': self.gtcounts[file_name[0]]
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

## Build Data Loader

In [9]:
dataset = MaizeTasselDataset(
        data_dir='maize_counting_dataset', 
        data_list='maize_counting_dataset/train.txt',
        ratio=0.167, 
        train=True, 
        transform=transforms.Compose([
            ToTensor()]
        )
    )

In [10]:
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        num_workers=0
    )

In [12]:
print(len(dataloader))
mean = 0.
std = 0.
for i, data in enumerate(dataloader, 0):
        images, targets = data['image'], data['target']
        bs = images.size(0)
        images = images.view(bs, images.size(1), -1).float()
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        print(f"index = {i} size = {images.size()}")
mean /= len(dataloader)
std /= len(dataloader)
print(mean/255.)
print(std/255.)

186
96.999985
index = 0 size = torch.Size([1, 3, 278770])
39.999996
index = 1 size = torch.Size([1, 3, 278770])
22.999998
index = 2 size = torch.Size([1, 3, 278770])
44.0
index = 3 size = torch.Size([1, 3, 278770])
32.0
index = 4 size = torch.Size([1, 3, 278770])
49.000004
index = 5 size = torch.Size([1, 3, 222530])
40.999992
index = 6 size = torch.Size([1, 3, 278770])
60.000004
index = 7 size = torch.Size([1, 3, 222530])
32.000004
index = 8 size = torch.Size([1, 3, 278770])
41.999996
index = 9 size = torch.Size([1, 3, 278770])
66.0
index = 10 size = torch.Size([1, 3, 278770])
65.99999
index = 11 size = torch.Size([1, 3, 278770])
63.0
index = 12 size = torch.Size([1, 3, 278770])
24.0
index = 13 size = torch.Size([1, 3, 278770])
39.999992
index = 14 size = torch.Size([1, 3, 278770])
36.000004
index = 15 size = torch.Size([1, 3, 278770])
42.0
index = 16 size = torch.Size([1, 3, 278770])
49.99999
index = 17 size = torch.Size([1, 3, 278770])
66.0
index = 18 size = torch.Size([1, 3, 278770]

40.000004
index = 155 size = torch.Size([1, 3, 278770])
5.0
index = 156 size = torch.Size([1, 3, 278770])
64.0
index = 157 size = torch.Size([1, 3, 278770])
120.0
index = 158 size = torch.Size([1, 3, 278770])
70.0
index = 159 size = torch.Size([1, 3, 278770])
60.999996
index = 160 size = torch.Size([1, 3, 278770])
24.000002
index = 161 size = torch.Size([1, 3, 222530])
66.00001
index = 162 size = torch.Size([1, 3, 278770])
0.0
index = 163 size = torch.Size([1, 3, 278770])
62.999996
index = 164 size = torch.Size([1, 3, 278770])
37.999992
index = 165 size = torch.Size([1, 3, 278770])
16.000002
index = 166 size = torch.Size([1, 3, 278770])
1.0
index = 167 size = torch.Size([1, 3, 278770])
16.0
index = 168 size = torch.Size([1, 3, 278770])
21.999994
index = 169 size = torch.Size([1, 3, 278770])
0.0
index = 170 size = torch.Size([1, 3, 278770])
40.0
index = 171 size = torch.Size([1, 3, 278770])
0.0
index = 172 size = torch.Size([1, 3, 278770])
14.000001
index = 173 size = torch.Size([1, 3, 

## Computation Functions

In [11]:
def compute_mae(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    mae = np.mean(np.abs(diff))
    return mae


def compute_mse(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    mse = np.sqrt(np.mean((diff ** 2)))
    return mse

def compute_relerr(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    diff = diff[gt > 0]
    gt = gt[gt > 0]
    if (diff is not None) and (gt is not None):
        rmae = np.mean(np.abs(diff) / gt) * 100
        rmse = np.sqrt(np.mean(diff**2 / gt**2)) * 100
    else:
        rmae = 0
        rmse = 0
    return rmae, rmse


def rsquared(pd, gt):
    """ Return R^2 where x and y are array-like."""
    pd, gt = np.array(pd), np.array(gt)
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(pd, gt)
    return r_value**2


def dense_sample2d(x, sx, stride):
    (h,w) = x.shape[:2]
    #idx_img = np.array([i for i in range(h*w)]).reshape(h,w)
    idx_img = np.zeros((h,w),dtype=float)
    
    th = [i for i in range(0, h-sx+1, stride)]
    tw = [j for j in range(0, w-sx+1, stride)]
    norm_vec = np.zeros(len(th)*len(tw))

    for i in th:
        for j in tw:
            idx_img[i:i+sx,j:j+sx] = idx_img[i:i+sx,j:j+sx]+1

    # plot redundancy map
    import os
    import matplotlib.pyplot as plt
    cmap = plt.cm.get_cmap('hot')
    idx_img = idx_img / (idx_img.max())
    idx_img = cmap(idx_img) * 255.
    plt.figure()
    plt.imshow(idx_img.astype(np.uint8))
    plt.axis('off')
    plt.savefig(os.path.join('redundancy_map.pdf'), bbox_inches='tight', dpi = 300)
    plt.close()
   
    idx_img = 1/idx_img
    idx_img = idx_img/sx/sx
    #line order
    idx = 0
    for i in th:
        for j in tw:
            norm_vec[idx] =idx_img[i:i+sx,j:j+sx].sum()
            idx+=1
    
    return norm_vec


def recover_countmap(pred, image, patch_sz, stride):
    pred = pred.reshape(-1)
    imH, imW = image.shape[2:4]
    cntMap = np.zeros((imH, imW), dtype=float)
    norMap = np.zeros((imH, imW), dtype=float)
    
    H = np.arange(0, imH - patch_sz + 1, stride)
    W = np.arange(0, imW - patch_sz + 1, stride)
    cnt = 0
    for h in H:
        for w in W:
            pixel_cnt = pred[cnt] / patch_sz / patch_sz
            cntMap[h:h+patch_sz, w:w+patch_sz] += pixel_cnt
            norMap[h:h+patch_sz, w:w+patch_sz] += np.ones((patch_sz,patch_sz))
            cnt += 1
    return cntMap / (norMap + 1e-12)

## Building the Model

In [12]:
class Encoder(nn.Module):
    def __init__(self, arc='tasselnetv2'):
        super(Encoder, self).__init__()
        if arc == 'tasselnetv2':
            self.encoder = nn.Sequential(
                nn.Conv2d(3, 16, 3, padding=1, bias=False),
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(16, 32, 3, padding=1, bias=False),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(32, 64, 3, padding=1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(64, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
            )
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.encoder(x)
        return x

In [13]:
class Counter(nn.Module):
    def __init__(self, arc='tasselnetv2', input_size=64, output_stride=8):
        super(Counter, self).__init__()
        k = int(input_size / 8)
        avg_pool_stride = int(output_stride / 8)

        if arc == 'tasselnetv2':
            self.counter = nn.Sequential(
                nn.Conv2d(128, 128, (k, k), bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 1, 1)
            )
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.counter(x)
        return x

In [14]:
class Normalizer:
    @staticmethod
    def cpu_normalizer(x, imh, imw, insz, os):
        # CPU normalization
        bs = x.size()[0]
        normx = np.zeros((imh, imw))
        norm_vec = dense_sample2d(normx, insz, os).astype(np.float32)
        x = x.cpu().detach().numpy().reshape(bs, -1) * norm_vec
        return x
    
    @staticmethod
    def gpu_normalizer(x, imh, imw, insz, os):
        _, _, h, w = x.size()            
        #accm = torch.cuda.FloatTensor(1, insz*insz, h*w).fill_(1)     
        accm = torch.FloatTensor(1, insz*insz, h*w).fill_(1) 
        accm = F.fold(accm, (imh, imw), kernel_size=insz, stride=os)
        accm = 1 / accm
        accm /= insz**2
        accm = F.unfold(accm, kernel_size=insz, stride=os).sum(1).view(1, 1, h, w)
        x *= accm
        return x.squeeze().cpu().detach().numpy()

In [15]:
class CountingModels(nn.Module):
    def __init__(self, arc='tasselnetv2', input_size=64, output_stride=8):
        super(CountingModels, self).__init__()
        self.input_size = input_size
        self.output_stride = output_stride

        self.encoder = Encoder(arc)
        self.counter = Counter(arc, input_size, output_stride)
        if arc == 'tasselnetv2':
          #changed
            self.normalizer = Normalizer.gpu_normalizer
        
        self.weight_init()

    def forward(self, x, is_normalize=True):
        imh, imw = x.size()[2:]
        x = self.encoder(x)
        x = self.counter(x)
        if is_normalize:
            x = self.normalizer(x, imh, imw, self.input_size, self.output_stride)
        return x

    def weight_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                # nn.init.kaiming_uniform_(
                #         m.weight, 
                #         mode='fan_in', 
                #         nonlinearity='relu'
                #         )
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

In [18]:
from time import time

insz, os = 64, 8
imH, imW = 1080, 1920
#net = CountingModels(arc='tasselnetv2', input_size=insz, output_stride=os).cuda()
net = CountingModels(arc='tasselnetv2', input_size=insz, output_stride=os)
with torch.no_grad():
    net.eval()
    #x = torch.randn(1, 3, imH, imW).cuda()
    x = torch.randn(1, 3, imH, imW)
    y = net(x)
    print(y.shape)    

with torch.no_grad():
    frame_rate = np.zeros((100, 1))
    for i in range(100):
        #x = torch.randn(1, 3, imH, imW).cuda()
        x = torch.randn(1, 3, imH, imW)
        #torch.cuda.synchronize()
        start = time()

        y = net(x)

        #torch.cuda.synchronize()
        end = time()

        running_frame_rate = 1 * float(1 / (end - start))
        frame_rate[i] = running_frame_rate
    print(np.mean(frame_rate))

(128, 233)
0.7050132465164394


In [19]:
y1 = np.argmax(y, axis=1)
y1

array([ 83,   7, 224, 213,  50,  19,  78,  11, 217,  71, 143, 113,  50,
       177, 126, 184,  39,  44, 100,  28, 153,  42, 167, 155,  30, 162,
       131, 224,  19,  42, 113, 195, 146,  36,  89, 114, 202, 117,  37,
        71, 179, 142, 107,  64,  39, 204, 112, 226, 182, 215, 214, 198,
        66, 160,  76,  88, 190,  77,  84, 158,  86, 203, 190,  86, 167,
       163,  82,  83, 207,  52, 126, 190, 106, 108, 134,  38,  39,  26,
        76,  57, 134, 215,  44, 179,  15, 169,  10, 128,  98, 150, 154,
       177,  35,  14, 203,  38,  80, 199, 180, 224,  62, 220,  59,  96,
        24,   9, 182,  44, 117, 116,  65, 129, 181, 193, 198,  44, 198,
        22, 147, 164,  94, 210,  86,  74, 186, 212,  46,  84], dtype=int64)

In [18]:
plt.switch_backend('agg')

In [17]:
# prevent dataloader deadlock, uncomment if deadlock occurs
cv2.setNumThreads(0)
#cudnn.enabled = True

# constant
IMG_SCALE = 1. / 255
IMG_MEAN = [.3405, .4747, .2418]
IMG_STD = [1, 1, 1]
SCALES = [0.7, 1, 1.3]
SHORTER_SIDE = 224

# system-related parameters
DATA_DIR = 'maize_counting_dataset'
DATASET = 'mtc'
EXP = 'exp_directory'
DATA_LIST = 'maize_counting_dataset/train.txt'
DATA_VAL_LIST = 'maize_counting_dataset/test.txt'

RESTORE_FROM = 'model_best.pth.tar'
SNAPSHOT_DIR = './snapshots'
RESULT_DIR = './results'

# model-related parameters
INPUT_SIZE = 64
OUTPUT_STRIDE = 8
MODEL = 'tasselnetv2'
RESIZE_RATIO = 0.125

# training-related parameters
OPTIMIZER = 'sgd'  # choice in ['sgd', 'adam']
BATCH_SIZE = 9
CROP_SIZE = (256, 256)
LEARNING_RATE = 1e-2
MILESTONES = [200, 400]
MOMENTUM = 0.95
MULT = 1
NUM_EPOCHS = 3
#NUM_EPOCHS = 500
NUM_CPU_WORKERS = 0
PRINT_EVERY = 1
RANDOM_SEED = 6
WEIGHT_DECAY = 5e-4
VAL_EVERY = 1

device = torch.device("cpu")
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# add a new entry here if creating a new data loader
dataset_list = {
    'mtc': MaizeTasselDataset
}

In [16]:
def save_checkpoint(state, snapshot_dir, filename='model_ckpt.pth.tar'):
    torch.save(state, '{}/{}'.format(snapshot_dir, filename))


def plot_learning_curves(net, dir_to_save):
    # plot learning curves
    fig = plt.figure(figsize=(16, 9))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(net.train_loss['epoch_loss'], label='train loss', color='tab:blue')
    ax1.legend(loc='upper right')
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(net.val_loss['epoch_loss'], label='val mae', color='tab:orange')
    ax2.legend(loc='upper right')
    # ax2.set_ylim((0,50))
    fig.savefig(os.path.join(dir_to_save, 'learning_curves.png'), bbox_inches='tight', dpi=300)
    plt.close()

## Train and Validate Methods

In [19]:
def train(net, train_loader, criterion, optimizer, epoch, args):
    # switch to 'train' mode
    net.train()

    # uncomment the following line if the training images don't have the same size
    #cudnn.benchmark = True

    if BATCH_SIZE == 1:
        for m in net.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    running_loss = 0.0
    avg_frame_rate = 0.0
    in_sz = INPUT_SIZE
    os = OUTPUT_STRIDE
    #target_filter = torch.cuda.FloatTensor(1, 1, in_sz, in_sz).fill_(1)
    target_filter = torch.FloatTensor(1, 1, in_sz, in_sz).fill_(1)
    for i, sample in enumerate(train_loader):
        #torch.cuda.synchronize()
        start = time()

        inputs, targets = sample['image'], sample['target']
        #inputs, targets = inputs.cuda(), targets.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = net(inputs, is_normalize=False)
        # generate targets
        targets = F.conv2d(targets, target_filter, stride=os)
        # compute loss
        loss = criterion(outputs, targets)

        # backward + optimize
        loss.backward()
        optimizer.step()
        # collect and print statistics
        running_loss += loss.item()

        #torch.cuda.synchronize()
        end = time()

        running_frame_rate = BATCH_SIZE * float(1 / (end - start))
        avg_frame_rate = (avg_frame_rate * i + running_frame_rate) / (i + 1)
        if i % PRINT_EVERY == PRINT_EVERY - 1:
            print('epoch: %d, train: %d/%d, '
                  'loss: %.5f, frame: %.2fHz/%.2fHz' % (
                      epoch,
                      i + 1,
                      len(train_loader),
                      running_loss / (i + 1),
                      running_frame_rate,
                      avg_frame_rate
                  ))
    net.train_loss['epoch_loss'].append(running_loss / (i + 1))

In [20]:
def validate(net, valset, val_loader, criterion, epoch, args):
    # switch to 'eval' mode
    net.eval()
    cudnn.benchmark = False

    image_list = valset.image_list

    #if args.save_output:
     #   epoch_result_dir = os.path.join(args.result_dir, str(epoch))
      #  if not os.path.exists(epoch_result_dir):
       #     os.makedirs(epoch_result_dir)
        #cmap = plt.cm.get_cmap('jet')
    epoch_result_dir = os.path.join(RESULT_DIR, str(epoch))
    if not os.path.exists(epoch_result_dir):
        os.makedirs(epoch_result_dir)
    cmap = plt.cm.get_cmap('jet')
    
    pd_counts = []
    gt_counts = []
    with torch.no_grad():
        avg_frame_rate = 0.0
        for i, sample in enumerate(val_loader):
            #torch.cuda.synchronize()
            start = time()

            image, gtcount = sample['image'], sample['gtcount']
            #inference
            #output = net(image.cuda(), is_normalize=not args["save_output"])
            output = net(image, is_normalize=not args["save_output"])
            #if args.save_output:
            #    output_save = output
                # normalization
            #    output = Normalizer.gpu_normalizer(output, image.size()[2], image.size()[3], args.input_size,
            #                                       args.output_stride)
            output_save = output
            output = Normalizer.gpu_normalizer(output, image.size()[2], image.size()[3], INPUT_SIZE,
                                                   OUTPUT_STRIDE)
            
            # postprocessing
            output = np.clip(output, 0, None)

            pdcount = output.sum()
            gtcount = float(gtcount.numpy())

            #if args.save_output:
            if args["save_output"]:
                _, image_name = os.path.split(image_list[i])
                output_save = np.clip(output_save.squeeze().cpu().numpy(), 0, None)
                output_save = recover_countmap(output_save, image, INPUT_SIZE, OUTPUT_STRIDE)
                output_save = output_save / (output_save.max() + 1e-12)
                output_save = cmap(output_save) * 255.
                # image composition
                image = valset.images[image_list[i]]
                nh, nw = output_save.shape[:2]
                image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)
                output_save = 0.5 * image + 0.5 * output_save[:, :, 0:3]

                dotimage = valset.dotimages[image_list[i]]

                fig = plt.figure()
                ax1 = fig.add_subplot(1, 2, 1)
                ax1.imshow(dotimage.astype(np.uint8))
                ax1.get_xaxis().set_visible(False)
                ax1.get_yaxis().set_visible(False)
                ax2 = fig.add_subplot(1, 2, 2)
                ax2.imshow(output_save.astype(np.uint8))
                ax2.get_xaxis().set_visible(False)
                ax2.get_yaxis().set_visible(False)
                fig.suptitle('manual count=%4.2f, inferred count=%4.2f' % (gtcount, pdcount), fontsize=10)
                if DATASET == 'mtc':
                    plt.tight_layout(rect=[0, 0, 1, 1.4])  # maize tassels counting
                plt.savefig(os.path.join(epoch_result_dir, image_name.replace('.jpg', '.png')), bbox_inches='tight',
                            dpi=300)
                plt.close()

            # compute mae and mse
            pd_counts.append(pdcount)
            gt_counts.append(gtcount)
            mae = compute_mae(pd_counts, gt_counts)
            mse = compute_mse(pd_counts, gt_counts)
            rmae, rmse = compute_relerr(pd_counts, gt_counts)

            #torch.cuda.synchronize()
            end = time()

            running_frame_rate = 1 * float(1 / (end - start))
            avg_frame_rate = (avg_frame_rate * i + running_frame_rate) / (i + 1)
            if i % PRINT_EVERY == PRINT_EVERY - 1:
                print(
                    'epoch: {0}, test: {1}/{2}, pre: {3:.2f}, gt:{4:.2f}, me:{5:.2f}, mae: {6:.2f}, mse: {7:.2f}, rmae: {8:.2f}%, rmse: {9:.2f}%, frame: {10:.2f}Hz/{11:.2f}Hz'
                        .format(epoch, i + 1, len(val_loader), pdcount, gtcount, pdcount - gtcount, mae, mse, rmae,
                                rmse, running_frame_rate, avg_frame_rate)
                )
            start = time()
    r2 = rsquared(pd_counts, gt_counts)
    np.save(SNAPSHOT_DIR + '/pd.npy', pd_counts)
    np.save(SNAPSHOT_DIR + '/gt.npy', gt_counts)
    print('epoch: {0}, mae: {1:.2f}, mse: {2:.2f}, rmae: {3:.2f}%, rmse: {4:.2f}%, r2: {5:.4f}'.format(epoch, mae, mse,
                                                                                                       rmae, rmse, r2))
    # write to files        
    with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
        print(
            'epoch: {0}, mae: {1:.2f}, mse: {2:.2f}, rmae: {3:.2f}%, rmse: {4:.2f}%, r2: {5:.4f}'.format(epoch, mae,
                                                                                                         mse, rmae,
                                                                                                         rmse, r2),
            file=f
        )
    with open(os.path.join(SNAPSHOT_DIR, 'counts.txt'), 'a') as f:
        for pd, gt in zip(pd_counts, gt_counts):
            print(
                '{0} {1}'.format(pd, gt),
                file=f
            )
    # save stats
    net.val_loss['epoch_loss'].append(mae)
    net.measure['mae'].append(mae)
    net.measure['mse'].append(mse)
    net.measure['rmae'].append(rmae)
    net.measure['rmse'].append(rmse)
    net.measure['r2'].append(r2)

In [21]:
args = {
    "image_scale":IMG_SCALE,
    "image_mean":IMG_MEAN,
    "image_std": IMG_STD,
    "scales":SCALES,
    "shorter_side":SHORTER_SIDE,
    "data_dir":DATA_DIR,
    "dataset":DATASET,
    "exp":EXP,
    "data_list":DATA_LIST,
    "data_val_list":DATA_VAL_LIST,
    "restore_from":RESTORE_FROM,
    "snapshot_dir":SNAPSHOT_DIR,
    "result_dir":RESULT_DIR,
    "save_output":"store_true",
    "input_size":INPUT_SIZE,
    "output_stride":OUTPUT_STRIDE,
    "resize_ratio":RESIZE_RATIO,
    "model":MODEL,
    "optimizer":OPTIMIZER,
    "batch_size":BATCH_SIZE,
    "milestones":MILESTONES,
    "crop_size":CROP_SIZE,
    "evaluate_only":"store_true",
    "learning_rate":LEARNING_RATE,
    "momentum":MOMENTUM,
    "weight_decay":WEIGHT_DECAY,
    "mult":MULT,
    "num_epochs":NUM_EPOCHS,
    "num_workers":NUM_CPU_WORKERS,
    "print_every":PRINT_EVERY,
    "random_seed":RANDOM_SEED,
    "val_every":VAL_EVERY
}

In [24]:
import os
IMG_MEAN = np.array(IMG_MEAN).reshape((1, 1, 3))
IMG_STD = np.array(IMG_STD).reshape((1, 1, 3))

CROP_SIZE = tuple(CROP_SIZE) if len(CROP_SIZE) > 1 else CROP_SIZE

# seeding for reproducbility
#if torch.cuda.is_available():
#    torch.cuda.manual_seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# instantiate dataset
dataset = dataset_list[DATASET]

SNAPSHOT_DIR = os.path.join(SNAPSHOT_DIR, DATASET.lower(), EXP)
if not os.path.exists(SNAPSHOT_DIR):
    os.makedirs(SNAPSHOT_DIR)

RESULT_DIR = os.path.join(RESULT_DIR, DATASET.lower(), EXP)
if not os.path.exists(RESULT_DIR):
    os.makedirs(RESULT_DIR)

RESTORE_FROM = os.path.join(SNAPSHOT_DIR, RESTORE_FROM)

#arguments = vars(args)
#for item in arguments:
    #print(item, ':\t', arguments[item])

In [22]:
# instantiate network
net = CountingModels(
      arc=MODEL,
      input_size=INPUT_SIZE,
      output_stride=OUTPUT_STRIDE
    )

net = nn.DataParallel(net)
#net.cuda()

# filter parameters
learning_params = [p[1] for p in net.named_parameters()]
pretrained_params = []

# define loss function and optimizer
#criterion = nn.L1Loss(reduction='mean').cuda()
criterion = nn.L1Loss(reduction='mean')

if OPTIMIZER == 'sgd':
    optimizer = torch.optim.SGD(
        [
            {'params': learning_params},
            {'params': pretrained_params, 'lr': LEARNING_RATE / MULT},
        ],
        lr=LEARNING_RATE,
        momentum=MOMENTUM,
        weight_decay=WEIGHT_DECAY
    )
elif OPTIMIZER == 'adam':
    optimizer = torch.optim.Adam(
        [
            {'params': learning_params},
            {'params': pretrained_params, 'lr': LEARNING_RATE / args.mult},
        ],
        lr=LEARNING_RATE
    )
else:
    raise NotImplementedError

In [25]:
# restore parameters
start_epoch = 0
net.train_loss = {
    'running_loss': [],
    'epoch_loss': []
}
net.val_loss = {
    'running_loss': [],
    'epoch_loss': []
}
net.measure = {
    'mae': [],
    'mse': [],
    'rmae': [],
    'rmse': [],
    'r2': []
}
if RESTORE_FROM is not None:
    if os.path.isfile(RESTORE_FROM):
        checkpoint = torch.load(RESTORE_FROM)
        net.load_state_dict(checkpoint['state_dict'])
        if 'epoch' in checkpoint:
            start_epoch = checkpoint['epoch']
        if 'optimizer' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
        if 'train_loss' in checkpoint:
            net.train_loss = checkpoint['train_loss']
        if 'val_loss' in checkpoint:
            net.val_loss = checkpoint['val_loss']
        if 'measure' in checkpoint:
            net.measure['mae'] = checkpoint['measure']['mae'] if 'mae' in checkpoint['measure'] else []
            net.measure['mse'] = checkpoint['measure']['mse'] if 'mse' in checkpoint['measure'] else []
            net.measure['rmae'] = checkpoint['measure']['rmae'] if 'rmae' in checkpoint['measure'] else []
            net.measure['rmse'] = checkpoint['measure']['rmse'] if 'rmse' in checkpoint['measure'] else []
            net.measure['r2'] = checkpoint['measure']['r2'] if 'r2' in checkpoint['measure'] else []
        print("==> load checkpoint '{}' (epoch {})"
              .format(RESTORE_FROM, start_epoch))
    else:
        with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
            for item in args:
                print(item, ':\t', args[item], file=f)
        print("==> no checkpoint found at '{}'".format(RESTORE_FROM))

==> load checkpoint './snapshots\mtc\exp_directory\model_best.pth.tar' (epoch 3)


In [26]:
# define transform
transform_train = [
    RandomCrop(CROP_SIZE),
    RandomFlip(),
    Normalize(
        IMG_SCALE,
        IMG_MEAN,
        IMG_STD
    ),
    ToTensor(),
    ZeroPadding(OUTPUT_STRIDE)
]
transform_val = [
    Normalize(
        IMG_SCALE,
        IMG_MEAN,
        IMG_STD
    ),
    ToTensor(),
    ZeroPadding(OUTPUT_STRIDE)
]
composed_transform_train = transforms.Compose(transform_train)
composed_transform_val = transforms.Compose(transform_val)

# define dataset loader
trainset = dataset(
    data_dir=DATA_DIR,
    data_list=DATA_LIST,
    ratio=RESIZE_RATIO,
    train=True,
    transform=composed_transform_train
)
train_loader = DataLoader(
    trainset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_CPU_WORKERS,
    pin_memory=True,
    drop_last=True
)
valset = dataset(
    data_dir=DATA_DIR,
    data_list=DATA_LIST,
    ratio=RESIZE_RATIO,
    train=False,
    transform=composed_transform_val
)
val_loader = DataLoader(
    valset,
    batch_size=1,
    shuffle=False,
    num_workers=NUM_CPU_WORKERS,
    pin_memory=True
)

## Test Run

In [ ]:
net.load_state_dict(torch.load('snapshots/mtc/path/model.pth'))

## Training the Model

In [30]:
print('alchemy start...')
#if evaluate_only:
#validate(net, valset, val_loader, criterion, start_epoch, args)
    #return

best_mae = 1000000.0
resume_epoch = -1 if start_epoch == 0 else start_epoch
scheduler = MultiStepLR(optimizer, milestones=MILESTONES, gamma=0.1, last_epoch=resume_epoch)
for epoch in range(start_epoch, NUM_EPOCHS):
    # train
    train(net, train_loader, criterion, optimizer, epoch + 1, args)
    if epoch % VAL_EVERY == VAL_EVERY - 1:
        # val
        validate(net, valset, val_loader, criterion, epoch + 1, args)
        # save_checkpoint
        state = {
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'train_loss': net.train_loss,
            'val_loss': net.val_loss,
            'measure': net.measure
        }
        save_checkpoint(state, SNAPSHOT_DIR, filename='model_ckpt.pth.tar')
        if net.measure['mae'][-1] <= best_mae:
            save_checkpoint(state, SNAPSHOT_DIR, filename='model_best.pth.tar')
            best_mae = net.measure['mae'][-1]
            best_mse = net.measure['mse'][-1]
            best_rmae = net.measure['rmae'][-1]
            best_rmse = net.measure['rmse'][-1]
            best_r2 = net.measure['r2'][-1]
        print(EXP + ' epoch {} finished!'.format(epoch + 1))
        print('best mae: {0:.2f}, best mse: {1:.2f}, best_rmae: {2:.2f}, best_rmse: {3:.2f}, best_r2: {4:.4f}'
              .format(best_mae, best_mse, best_rmae, best_rmse, best_r2))
        plot_learning_curves(net, SNAPSHOT_DIR)
    scheduler.step()

print('Experiments with ' + EXP + ' done!')
with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
    print(
        'best mae: {0:.2f}, best mse: {1:.2f}, best_rmae: {2:.2f}, best_rmse: {3:.2f}, best_r2: {4:.4f}'
            .format(best_mae, best_mse, best_rmae, best_rmse, best_r2),
        file=f
    )
    print(
        'overall best mae: {0:.2f}, overall best mse: {1:.2f}, overall best_rmae: {2:.2f}, overall best_rmse: {3:.2f}, overall best_r2: {4:.4f}'
            .format(min(net.measure['mae']), min(net.measure['mse']), min(net.measure['rmae']),
                    min(net.measure['rmse']), max(net.measure['r2'])),
        file=f
    )

alchemy start...
57.0
8 8
16.0
8 8
23.0
8 8
49.0
8 8
18.0
8 8
32.0
8 8
65.00001
8 8
14.000002
8 8
67.0
8 8
epoch: 3, train: 1/20, loss: 0.41205, frame: 17.28Hz/17.28Hz
42.000008
8 8
1.0000001
8 8
9.0
8 8
51.000004
8 8
65.99999
8 8
104.00001
8 8
37.000004
8 8
64.00001
8 8
0.0
8 8
epoch: 3, train: 2/20, loss: 0.49046, frame: 17.55Hz/17.42Hz
18.000002
8 8
44.0
8 8
2.0
8 8
85.00001
8 8
17.000002
8 8
38.0
8 8
56.000004
8 8
62.999996
8 8
63.999992
8 8
epoch: 3, train: 3/20, loss: 0.50815, frame: 17.06Hz/17.30Hz
39.999992
8 8
120.0
8 8
69.0
8 8
27.999998
8 8
54.0
8 8
36.000004
8 8
15.999999
8 8
51.0
8 8
19.0
8 8
epoch: 3, train: 4/20, loss: 0.52089, frame: 17.15Hz/17.26Hz
0.0
8 8
68.00001
8 8
38.0
8 8
62.999996
8 8
18.0
8 8
65.99999
8 8
60.0
8 8
70.0
8 8
0.0
8 8
epoch: 3, train: 5/20, loss: 0.51547, frame: 16.55Hz/17.12Hz
62.0
8 8
69.0
8 8
0.0
8 8
30.999998
8 8
41.999996
8 8
1.0
8 8
50.999996
8 8
5.0
8 8
2.9999998
8 8
epoch: 3, train: 6/20, loss: 0.48653, frame: 17.38Hz/17.16Hz
66.0
8 8
54.0


9.0
2 8
[0, 0, 1, 1]
epoch: 3, test: 34/186, pre: 21.75, gt:9.00, me:12.75, mae: 14.60, mse: 19.80, rmae: 36.79%, rmse: 52.92%, frame: 0.99Hz/1.01Hz
30.999998
2 8
[0, 0, 1, 1]
epoch: 3, test: 35/186, pre: 45.38, gt:31.00, me:14.38, mae: 14.60, mse: 19.66, rmae: 37.08%, rmse: 52.74%, frame: 1.04Hz/1.01Hz
5.0
2 8
[0, 0, 1, 1]
epoch: 3, test: 36/186, pre: 12.85, gt:5.00, me:7.85, mae: 14.41, mse: 19.43, rmae: 40.61%, rmse: 58.52%, frame: 1.01Hz/1.01Hz
39.0
2 8
[0, 0, 1, 1]
epoch: 3, test: 37/186, pre: 32.07, gt:39.00, me:-6.93, mae: 14.21, mse: 19.20, rmae: 39.96%, rmse: 57.76%, frame: 0.98Hz/1.01Hz
94.00001
2 8
[0, 0, 1, 1]
epoch: 3, test: 38/186, pre: 42.59, gt:94.00, me:-51.41, mae: 15.19, mse: 20.70, rmae: 40.37%, rmse: 57.68%, frame: 0.92Hz/1.01Hz
37.0
2 8
[0, 0, 1, 1]
epoch: 3, test: 39/186, pre: 32.30, gt:37.00, me:-4.70, mae: 14.92, mse: 20.45, rmae: 39.62%, rmse: 56.93%, frame: 0.94Hz/1.01Hz
68.00001
2 8
[0, 0, 1, 1]
epoch: 3, test: 40/186, pre: 34.68, gt:68.00, me:-33.32, mae: 1

116.99999
2 8
[0, 0, 1, 1]
epoch: 3, test: 88/186, pre: 49.64, gt:117.00, me:-67.36, mae: 14.60, mse: 20.21, rmae: 40.97%, rmse: 79.13%, frame: 0.92Hz/1.02Hz
0.0
2 8
[0, 0, 1, 1]
epoch: 3, test: 89/186, pre: 6.53, gt:0.00, me:6.53, mae: 14.50, mse: 20.10, rmae: 40.97%, rmse: 79.13%, frame: 0.98Hz/1.02Hz
0.0
2 8
[0, 0, 1, 1]
epoch: 3, test: 90/186, pre: 3.64, gt:0.00, me:3.64, mae: 14.38, mse: 20.00, rmae: 40.97%, rmse: 79.13%, frame: 0.98Hz/1.02Hz
62.999996
2 8
[0, 0, 1, 1]
epoch: 3, test: 91/186, pre: 45.46, gt:63.00, me:-17.54, mae: 14.42, mse: 19.97, rmae: 40.81%, rmse: 78.71%, frame: 1.03Hz/1.02Hz
2.0
2 8
[0, 0, 1, 1]
epoch: 3, test: 92/186, pre: 3.54, gt:2.00, me:1.54, mae: 14.28, mse: 19.86, rmae: 41.25%, rmse: 78.68%, frame: 0.98Hz/1.02Hz
1.9999998
8 8
epoch: 3, test: 93/186, pre: 1.49, gt:2.00, me:-0.51, mae: 14.13, mse: 19.75, rmae: 41.06%, rmse: 78.27%, frame: 1.17Hz/1.02Hz
62.000004
2 8
[0, 0, 1, 1]
epoch: 3, test: 94/186, pre: 42.44, gt:62.00, me:-19.56, mae: 14.19, mse: 19

96.0
2 8
[0, 0, 1, 1]
epoch: 3, test: 142/186, pre: 57.30, gt:96.00, me:-38.70, mae: 13.35, mse: 18.70, rmae: 51.35%, rmse: 116.55%, frame: 0.99Hz/1.02Hz
17.999998
2 8
[0, 0, 1, 1]
epoch: 3, test: 143/186, pre: 15.82, gt:18.00, me:-2.18, mae: 13.28, mse: 18.64, rmae: 51.06%, rmse: 116.12%, frame: 0.95Hz/1.02Hz
99.99999
2 8
[0, 0, 1, 1]
epoch: 3, test: 144/186, pre: 52.15, gt:100.00, me:-47.85, mae: 13.52, mse: 19.00, rmae: 51.03%, rmse: 115.76%, frame: 0.95Hz/1.02Hz
63.999996
2 8
[0, 0, 1, 1]
epoch: 3, test: 145/186, pre: 43.35, gt:64.00, me:-20.65, mae: 13.57, mse: 19.01, rmae: 50.89%, rmse: 115.36%, frame: 1.01Hz/1.02Hz
48.0
2 8
[0, 0, 1, 1]
epoch: 3, test: 146/186, pre: 48.40, gt:48.00, me:0.40, mae: 13.48, mse: 18.94, rmae: 50.53%, rmse: 114.94%, frame: 1.01Hz/1.01Hz
5.0000005
2 8
[0, 0, 1, 1]
epoch: 3, test: 147/186, pre: 14.59, gt:5.00, me:9.59, mae: 13.45, mse: 18.89, rmae: 51.56%, rmse: 115.69%, frame: 1.01Hz/1.01Hz
64.99999
2 8
[0, 0, 1, 1]
epoch: 3, test: 148/186, pre: 53.36,

## Validate the Model

In [31]:
validate(net, valset, val_loader, criterion, start_epoch, args)

2 8
[0, 0, 1, 1]
epoch: 2, test: 1/186, pre: 51.48, gt:97.00, me:-45.52, mae: 45.52, mse: 45.52, rmae: 46.93%, rmse: 46.93%, frame: 0.82Hz/0.82Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 2/186, pre: 31.72, gt:40.00, me:-8.28, mae: 26.90, mse: 32.71, rmae: 33.81%, rmse: 36.26%, frame: 1.02Hz/0.92Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 3/186, pre: 22.57, gt:23.00, me:-0.43, mae: 18.08, mse: 26.71, rmae: 23.17%, rmse: 29.63%, frame: 1.02Hz/0.95Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 4/186, pre: 36.04, gt:44.00, me:-7.96, mae: 15.55, mse: 23.47, rmae: 21.90%, rmse: 27.21%, frame: 1.03Hz/0.97Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 5/186, pre: 25.47, gt:32.00, me:-6.53, mae: 13.74, mse: 21.20, rmae: 21.60%, rmse: 25.99%, frame: 1.02Hz/0.98Hz
8 8
epoch: 2, test: 6/186, pre: 12.10, gt:49.00, me:-36.90, mae: 17.60, mse: 24.52, rmae: 30.56%, rmse: 38.84%, frame: 1.12Hz/1.00Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 7/186, pre: 32.07, gt:41.00, me:-8.93, mae: 16.37, mse: 22.95, rmae: 29.30%, rmse: 36.89%, frame: 1.05Hz/1.0

epoch: 2, test: 58/186, pre: 40.58, gt:65.00, me:-24.42, mae: 14.83, mse: 20.04, rmae: 45.63%, rmse: 93.88%, frame: 1.03Hz/1.01Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 59/186, pre: 22.04, gt:21.00, me:1.04, mae: 14.60, mse: 19.87, rmae: 44.87%, rmse: 92.99%, frame: 0.99Hz/1.01Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 60/186, pre: 43.00, gt:52.00, me:-9.00, mae: 14.50, mse: 19.74, rmae: 44.36%, rmse: 92.16%, frame: 0.95Hz/1.01Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 61/186, pre: 43.11, gt:51.00, me:-7.89, mae: 14.40, mse: 19.61, rmae: 43.83%, rmse: 91.34%, frame: 1.05Hz/1.01Hz
8 8
epoch: 2, test: 62/186, pre: 18.85, gt:36.00, me:-17.15, mae: 14.44, mse: 19.57, rmae: 43.90%, rmse: 90.74%, frame: 1.21Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 63/186, pre: 6.87, gt:0.00, me:6.87, mae: 14.32, mse: 19.43, rmae: 43.90%, rmse: 90.74%, frame: 1.04Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 64/186, pre: 2.20, gt:1.00, me:1.20, mae: 14.11, mse: 19.28, rmae: 45.24%, rmse: 91.34%, frame: 1.00Hz/1.02Hz
2 8
[0, 0, 1,

epoch: 2, test: 115/186, pre: 45.31, gt:56.00, me:-10.69, mae: 14.01, mse: 19.33, rmae: 44.00%, rmse: 89.20%, frame: 1.03Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 116/186, pre: 13.46, gt:15.00, me:-1.54, mae: 13.90, mse: 19.25, rmae: 43.68%, rmse: 88.78%, frame: 0.99Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 117/186, pre: 44.16, gt:52.00, me:-7.84, mae: 13.85, mse: 19.18, rmae: 43.42%, rmse: 88.38%, frame: 1.04Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 118/186, pre: 56.32, gt:47.00, me:9.32, mae: 13.81, mse: 19.12, rmae: 43.20%, rmse: 87.99%, frame: 1.01Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 119/186, pre: 46.45, gt:51.00, me:-4.55, mae: 13.73, mse: 19.04, rmae: 42.88%, rmse: 87.59%, frame: 1.00Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 120/186, pre: 40.95, gt:41.00, me:-0.05, mae: 13.62, mse: 18.96, rmae: 42.49%, rmse: 87.19%, frame: 0.94Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 121/186, pre: 23.43, gt:45.00, me:-21.57, mae: 13.69, mse: 18.99, rmae: 42.54%, rmse: 86.91%, frame: 1

epoch: 2, test: 171/186, pre: 28.67, gt:0.00, me:28.67, mae: 13.12, mse: 18.64, rmae: 50.26%, rmse: 112.40%, frame: 0.98Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 172/186, pre: 50.76, gt:40.00, me:10.76, mae: 13.10, mse: 18.60, rmae: 50.12%, rmse: 112.07%, frame: 1.03Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 173/186, pre: 26.12, gt:0.00, me:26.12, mae: 13.18, mse: 18.65, rmae: 50.12%, rmse: 112.07%, frame: 0.99Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 174/186, pre: 14.34, gt:14.00, me:0.34, mae: 13.10, mse: 18.60, rmae: 49.82%, rmse: 111.72%, frame: 1.00Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 175/186, pre: 30.41, gt:54.00, me:-23.59, mae: 13.16, mse: 18.63, rmae: 49.78%, rmse: 111.43%, frame: 1.00Hz/1.02Hz
8 8
epoch: 2, test: 176/186, pre: 25.11, gt:64.00, me:-38.89, mae: 13.31, mse: 18.81, rmae: 49.85%, rmse: 111.18%, frame: 1.23Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 2, test: 177/186, pre: 39.22, gt:41.00, me:-1.78, mae: 13.24, mse: 18.76, rmae: 49.57%, rmse: 110.84%, frame: 1.01Hz/1.